In [1]:
"""
File v723_optima_increase_ci.py

Michel Bierlaire
Mon Aug 31 15:20:02 2020

"""
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta

# The calculation of confidence intervals is based on
# simulation. Therefore, each run will report a different output due
# to the generation of random numbers.  In order to obtain the same
# output as in the report, we set the seed for the random number
# generation procedure.

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('boeing', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5
male = Gender == 1
female = Gender == 2
unreportedGender = Gender == -1
fulltime = OccupStat == 1
notfulltime = OccupStat != 1

# Removing some observations
exclude = ((Choice == -1) + ((CarAvail == 3) * (Choice == 1))) != 0
database.remove(exclude)

# List of parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_TIME_FULLTIME = Beta('BETA_TIME_FULLTIME', 0, None, None, 0)
BETA_TIME_OTHER = Beta('BETA_TIME_OTHER', 0, None, None, 0)
BETA_DIST_MALE = Beta('BETA_DIST_MALE', 0, None, None, 0)
BETA_DIST_FEMALE = Beta('BETA_DIST_FEMALE', 0, None, None, 0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)

# We define a function that takes the increase of gas cost as input
# and returns the market shares

def calculateMarketShares(increase):
    # Definition of utility functions:
    V_PT = BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
        BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
        BETA_COST * MarginalCostPT_scaled
    V_CAR = ASC_CAR + \
        BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
        BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
        BETA_COST * CostCarCHF_scaled * (1 + increase)
    V_SM = ASC_SM + \
        BETA_DIST_MALE * distance_km_scaled * male + \
        BETA_DIST_FEMALE * distance_km_scaled * female + \
        BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

    # Associate utility functions with the numbering of alternatives
    V = {0: V_PT,
         1: V_CAR,
         2: V_SM}

    try:
        m = 'v715_optima_model'
        results = res.bioResults(pickleFile=f'{m}.pickle')
    except FileNotFoundError:
        print(f'Run first the script {m}.py in order to generate the file '
              '{m}.pickle.')
        sys.exit()

    prob_pt = models.logit(V, None, 0)
    prob_car = models.logit(V, None, 1)
    prob_sm = models.logit(V, None, 2)

    simulate = {'Prob. public transp.': prob_pt,
                'Prob. car': prob_car,
                'Prob. slow modes': prob_sm,
                'Weight': Weight}

    biosim = bio.BIOGEME(database, simulate)
    biosim.modelName = 'v715_optima_simul'

    # Perform the simulation
    simresults = biosim.simulate(results.getBetaValues())

    # Calculate confidence intervals
    b = results.getBetasForSensitivityAnalysis(biosim.freeBetaNames, size=1000)
    left, right = biosim.confidenceIntervals(b, 0.9)

    totalWeight = simresults['Weight'].sum()

    simresults['Weighted public'] = simresults['Prob. public transp.'] * simresults['Weight']
    left['Weighted public'] = left['Prob. public transp.'] * left['Weight']
    right['Weighted public'] = right['Prob. public transp.'] * right['Weight']
    simresults['Weighted car'] = simresults['Prob. car'] * simresults['Weight']
    left['Weighted car'] = left['Prob. car'] * left['Weight']
    right['Weighted car'] = right['Prob. car'] * right['Weight']
    simresults['Weighted slow'] = simresults['Prob. slow modes'] * simresults['Weight']
    left['Weighted slow'] = left['Prob. slow modes'] * left['Weight']
    right['Weighted slow'] = right['Prob. slow modes'] * right['Weight']

    # Market shares are tuple containing
    #      0. the left extremity of the confidence interval,
    #      1. the simulated result,
    #      2. the right extremity of the confidence interval.
    pt = (left['Weighted public'].sum() / totalWeight,
             simresults['Weighted public'].sum() / totalWeight,
             right['Weighted public'].sum() / totalWeight)
    car = (left['Weighted car'].sum() / totalWeight,
              simresults['Weighted car'].sum() / totalWeight,
              right['Weighted car'].sum() / totalWeight)
    sm = (left['Weighted slow'].sum() / totalWeight,
             simresults['Weighted slow'].sum() / totalWeight,
             right['Weighted slow'].sum() / totalWeight)
    return pt, car, sm

for i in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]:
    ms_pt, ms_car, ms_sm = calculateMarketShares(i)
    print(f'Market shares with increase {100*i:2.0f}%:\t'
          f'PT {100*ms_pt[1]:.2f}% [{100*ms_pt[0]:.1f}%:{100*ms_pt[2]:.1f}%], '
          f'car {100*ms_car[1]:.2f}%[{100*ms_car[0]:.1f}%:{100*ms_car[2]:.1f}%], '
          f'slow modes {100*ms_sm[1]:.2f}% [{100*ms_sm[0]:.1f}%:{100*ms_sm[2]:.1f}%]')

Market shares with increase  0%:	PT 28.70% [25.1%:32.3%], car 65.22%[60.6%:69.0%], slow modes 6.08% [4.1%:9.6%]
Market shares with increase  5%:	PT 29.00% [25.3%:32.7%], car 64.91%[60.1%:68.8%], slow modes 6.10% [4.1%:9.7%]
Market shares with increase 10%:	PT 29.30% [25.5%:32.9%], car 64.59%[60.0%:68.5%], slow modes 6.11% [4.2%:9.6%]
Market shares with increase 15%:	PT 29.60% [25.9%:33.3%], car 64.28%[59.6%:68.2%], slow modes 6.12% [4.2%:9.6%]
Market shares with increase 20%:	PT 29.90% [26.2%:33.6%], car 63.97%[59.2%:67.9%], slow modes 6.14% [4.1%:9.6%]
Market shares with increase 25%:	PT 30.20% [26.4%:33.9%], car 63.65%[58.8%:67.6%], slow modes 6.15% [4.1%:9.9%]
Market shares with increase 30%:	PT 30.50% [26.7%:34.3%], car 63.33%[58.5%:67.3%], slow modes 6.17% [4.2%:9.6%]
